In [1]:
import numpy as np
import pandas as pd
import warnings
import os
from tqdm import tqdm
from sklearn import preprocessing, metrics
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [2]:
df_feature = pd.read_pickle('./temp/part1_feature.plk')
df_distance = pd.read_pickle('./temp/distance.plk')
#df_feature.head()

,courier_id,wave_index,tracking_id,courier_wave_start_lng,courier_wave_start_lat,action_type,expect_time,date,type,target,group,id,current_time,last_tracking_id,last_action_type,source_lng,source_lat,target_lng,target_lat,grid_distance,weather_grade,aoi_id,shop_id,promise_deliver_time,estimate_pick_time,level,speed,max_load
0,10007871,0,2100074550065333539,121.630997,39.142343,DELIVERY,1580528963,20200201,train,1.0,20200201100078710,0,1580528622,2100074550065333539,PICKUP,121.631219,39.141811,121.632084,39.146201,707.0,正常天气,b71df7214347524a0f5f0c79dfdf2f4e,88ac051764fe348382e6529948de8015,1580530276,1580529019,3,4.751832,11
1,10007871,0,2100074550779577850,121.630997,39.142343,PICKUP,1580529129,20200201,train,0.0,20200201100078710,1,1580528622,2100074550065333539,PICKUP,121.631219,39.141811,121.631574,39.142231,152.0,正常天气,753c1911e8a294c5db901f8555faff0c,92ec52685bd511da262ee6f7a0adaa87,1580530236,1580529399,3,4.751832,11
2,10007871,0,2100074550779577850,121.630997,39.142343,DELIVERY,1580529444,20200201,train,0.0,20200201100078710,2,1580528622,2100074550065333539,PICKUP,121.631219,39.141811,121.635154,39.143561,671.0,正常天气,753c1911e8a294c5db901f8555faff0c,92ec52685bd511da262ee6f7a0adaa87,1580530236,1580529399,3,4.751832,11
3,10007871,1,2100074555638285402,121.631208,39.142519,PICKUP,1580532225,20200201,train,1.0,20200201100078711,3,1580532113,2100074554932692192,DELIVERY,121.636904,39.142721,121.636701,39.141801,160.0,正常天气,898aef0932f6aaecda27aba8e9903991,1af41a72adbdb2abd7a2dab03e357bcf,1580533463,1580532384,3,4.751832,11
4,10007871,1,2100074554118800474,121.631208,39.142519,PICKUP,1580532227,20200201,train,0.0,20200201100078711,4,1580532113,2100074554932692192,DELIVERY,121.636904,39.142721,121.636701,39.141801,160.0,正常天气,0d109cc0999fa3a108c67cb748b1931f,1af41a72adbdb2abd7a2dab03e357bcf,1580533598,1580532339,3,4.751832,11


In [3]:
#添加distance2
#featureGrouped=df_feature.groupby('group')
#定义函数，获取originList，distanceList
def get_lists(distance_grouped):
    #得到两个Dataframe
    #f=featureGrouped.get_group(group)
    for group,name in distance_grouped:
        d=distance_grouped.get_group(group)
        #
        #f=f[['tracking_id','action_type']]
        d=d[['tracking_id','source_type','target_tracking_id','target_type','grid_distance']]
        #originList=np.array(f).to_list()
        distanceList=np.array(d).tolist()
    return distanceList  #originList,     #################

# findMin函数
def findMin(originList,distanceList):
    listS=permute(originList)
    listS=checkListS(listS)
    distances=[]
    for x in listS:
        sumD=0
        for i in range(len(x)-1):
            sumD=sumD+findDistance([x[i],x[i+1]],distanceList)    #####
        distances.append(sumD)
    return min(distances)


#查找距离，输入为[[id,type],[id,type]]distanceList
def findDistance(listTT,distanceList):
    for x in range(len(distanceList)):
        if [listTT[0][0],listTT[0][1],listTT[1][0],listTT[1][1]]==distanceList[:-1]:
            return distanceList[-1].astype('int')

#进行排列检查,其实只需查找是否dilevery在pickup前面
def checkS(S):
    for i in range(len(S)):
        if S[i][1]=='PICKUP':
            if [S[i][0],'DELIVERY'] in S[:i]:
                return False
    return True

    
def checkListS(listS):
    trueS=[]
    for x in listS:
        if checkS(x):
            trueS.append(x)
    return trueS
def permute(nums):
        from itertools import permutations
        result = []
        for i in permutations(nums,len(nums)):
            result.append(list(i))
        return result

In [4]:
#distance在这里操作
df_distance0=df_distance
del df_distance
distance_grouped=df_distance0.groupby('group')
distanceList0 = get_lists(distance_grouped)

df_next_action_pred = df_feature[['courier_id','wave_index','tracking_id','action_type','group']]
del df_feature
#df_next_action_pred.head()
group_index = df_next_action_pred.groupby(['courier_id','wave_index','group'])
    #print('name:')
    #print (name)
    #print('group:')
    #print (group)
    
#df_picked_list = []
#df_unpicked_list = []  
for group, name in group_index:
    unsorted_list = np.array(name[['tracking_id','action_type']])
    unsorted_list = unsorted_list.tolist()
    #print(unsorted_list)
    #result = permute(unsorted_list)
    
    result = findMin(unsorted_list,distanceList0)
    
    #del unsorted_list
    #print (result)
    #print(type(result))
    #print(np.array(result).shape)
    #print(result[0][0])
    #S = (result[:][:][2],result[:;:;3])
    #print (result)
    #result = checkListS(result)
    
    
    #print('@@@@@@')
    #print (result)

    #name_grouped = name.groupby(['tracking_id'])
    #df_picked_data = []
    #df_unpicked_data = []
    #for group, name in name_grouped:

        #print (name.shape)
        #if name.shape[0] < 2:
            #df_picked_data = name 
        #else:
            #df_unpicked_data = name
        #if not (df_picked_data):
        #print(type(df_picked_data))
        #elif df_unpicked_data != 0
        #break
    #print (df_picked_data)
    #print (df_unpicked_data[1])
    break
        



TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'